# PineconeEmbeddings

This will help you get started with PineconeEmbeddings [embedding models](/docs/concepts/embedding_models) using LangChain. For detailed documentation on `PineconeEmbeddings` features and configuration options, please refer to the [API reference](https://api.js.langchain.com/classes/_langchain_pinecone.PineconeEmbeddings.html).

## Overview
### Integration details

| Class | Package | Local | [Py support](https://python.langchain.com/docs/integrations/text_embedding/pinecone/) | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [PineconeEmbeddings](https://api.js.langchain.com/classes/_langchain_pinecone.PineconeEmbeddings.html) | [@langchain/pinecone](https://api.js.langchain.com/classes/_langchain_pinecone.PineconeEmbeddings.html) | ❌ | ✅ | ![NPM - Downloads](https://img.shields.io/npm/dm/@langchain/pinecone?style=flat-square&label=%20&) | ![NPM - Version](https://img.shields.io/npm/v/@langchain/pinecone?style=flat-square&label=%20&) |

## Setup

To access Pinecone embedding models you'll need to create a Pinecone account, get an API key, and install the `@langchain/pinecone` integration package.

### Credentials

Sign up for a [Pinecone](https://www.pinecone.io/) account, retrieve your API key, and set it as an environment variable named `PINECONE_API_KEY`:

```typescript
process.env.PINECONE_API_KEY = "your-pinecone-api-key";
```

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```bash
# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="your-api-key"
```

### Installation

The LangChain PineconeEmbeddings integration lives in the `@langchain/pinecone` package:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/pinecone @langchain/core @pinecone-database/pinecone@5
</Npm2Yarn>
```

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [1]:
import { PineconeEmbeddings } from "@langchain/pinecone";

const embeddings = new PineconeEmbeddings({
  model: "multilingual-e5-large",
});

## Indexing and Retrieval

Embedding models are often used in retrieval-augmented generation (RAG) flows, both as part of indexing data as well as later retrieving it. For more detailed instructions, please see our RAG tutorials under the [working with external knowledge tutorials](/docs/tutorials/#working-with-external-knowledge).

Below, see how to index and retrieve data using the `embeddings` object we initialized above. In this example, we will index and retrieve a sample document using the demo [`MemoryVectorStore`](/docs/integrations/vectorstores/memory).

In [2]:
// Create a vector store with a sample text
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const text = "LangChain is the framework for building context-aware reasoning applications";

const vectorstore = await MemoryVectorStore.fromDocuments(
  [{ pageContent: text, metadata: {} }],
  embeddings,
);

// Use the vector store as a retriever that returns a single document
const retriever = vectorstore.asRetriever(1);

// Retrieve the most similar text
const retrievedDocuments = await retriever.invoke("What is LangChain?");

retrievedDocuments[0].pageContent;

LangChain is the framework for building context-aware reasoning applications


## Direct Usage

Under the hood, the vectorstore and retriever implementations are calling `embeddings.embedDocument(...)` and `embeddings.embedQuery(...)` to create embeddings for the text(s) used in `fromDocuments` and the retriever's `invoke` operations, respectively.

You can directly call these methods to get embeddings for your own use cases.

### Embed single texts

You can embed queries for search with `embedQuery`. This generates a vector representation specific to the query:

In [3]:
const singleVector = await embeddings.embedQuery(text);

console.log(singleVector.slice(0, 100));

[
         0.0191650390625,  0.004924774169921875,     -0.015838623046875,
          -0.04248046875,     0.040191650390625,      -0.02679443359375,
     -0.0240936279296875,     0.058624267578125,      0.027069091796875,
        -0.0435791015625,      0.01934814453125,      0.040191650390625,
     -0.0194244384765625,   0.01386260986328125,    -0.0216827392578125,
    -0.01073455810546875,   -0.0166168212890625,    0.01073455810546875,
        -0.0228271484375,       0.0062255859375,      0.035064697265625,
     -0.0114593505859375,   -0.0257110595703125,    -0.0285797119140625,
        0.01190185546875,    -0.022186279296875,   -0.01500701904296875,
       -0.03240966796875, 0.0019063949584960938,     -0.039337158203125,
     -0.0047454833984375,              -0.03125,       -0.0123291015625,
    -0.00899505615234375,        -0.02880859375,      0.014678955078125,
         0.0452880859375,      0.05035400390625,     -0.053436279296875,
      0.0265960693359375,   -0.0206756591796875, 

### Embed multiple texts

You can embed multiple texts for indexing with `embedDocuments`. The internals used for this method may (but do not have to) differ from embedding queries:

In [4]:
const text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs";

const vectors = await embeddings.embedDocuments([text, text2]);

console.log(vectors[0].slice(0, 100));
console.log(vectors[1].slice(0, 100));

[
      0.0190887451171875,  0.00482940673828125,   -0.0158233642578125,
       -0.04254150390625,    0.040130615234375,   -0.0268096923828125,
          -0.02392578125,    0.058624267578125,    0.0269927978515625,
          -0.04345703125,   0.0193328857421875,     0.040374755859375,
     -0.0196075439453125,  0.01384735107421875,    -0.021881103515625,
    -0.01068878173828125,   -0.016510009765625,   0.01079559326171875,
     -0.0227813720703125,        0.00634765625,     0.035064697265625,
     -0.0113983154296875,  -0.0257720947265625,   -0.0285491943359375,
       0.011749267578125,  -0.0222625732421875,   -0.0148468017578125,
        -0.0325927734375,  0.00203704833984375,      -0.0394287109375,
   -0.004878997802734375,  -0.0311126708984375,  -0.01248931884765625,
    -0.00897979736328125,  -0.0286407470703125,    0.0146331787109375,
        0.04522705078125,    0.050201416015625,    -0.053314208984375,
      0.0265960693359375,  -0.0207366943359375,      0.06658935546875,
    

## API reference

For detailed documentation of all PineconeEmbeddings features and configurations head to the API reference: https://api.js.langchain.com/classes/_langchain_pinecone.PineconeEmbeddings.html